In [194]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import together
from time import sleep
import re
from sklearn.metrics import mean_absolute_error
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [195]:
def dprint(s, debug):
    if debug:
        print(s)

In [196]:
# TODO: find your API key here
# https://api.together.xyz/settings/api-keys
YOUR_API_KEY = '5a70c1d03697cbf9d7a2c4595ffa8f892bc25311ee89de4dceda77eefb228a84'
together.api_key = YOUR_API_KEY

def call_together_api(prompt, student_configs, pre_processing, post_processing, model='togethercomputer/llama-2-70b', debug=False):
    prompt = pre_processing(prompt)
    output = together.Complete.create(
    prompt = prompt,
    model = model, 
    **student_configs
    )
    dprint('*****prompt*****', debug)
    dprint(prompt, debug)
    dprint('*****result*****', debug)
    res = output['output']['choices'][0]['text']
    dprint(res, debug)
    dprint('*****output*****', debug)
    labels_only = post_processing(res)
    dprint('POST PROCESSED', debug)
    dprint(labels_only, debug)
    dprint('=========', debug)
    return labels_only

In [197]:
model_names = [
    'togethercomputer/llama-2-7b', #LLaMa-2-7B
    'togethercomputer/llama-2-13b', #LLaMa-2-13B
    'togethercomputer/llama-2-70b', #LLaMa-2-70B
    'togethercomputer/llama-2-70b-chat', #LLaMa-2-70B-Chat
]

In [198]:
def get_train_df(topn = 10):
    train_df = pd.read_csv('./data/splits/train.csv')
    return train_df[:topn]


In [199]:
def get_test_set():
    test_df = pd.read_csv('./data/splits/test.csv')
    return test_df
    

In [200]:
def get_eval_df(topn = 5):
    eval_df = pd.read_csv('./data/splits/val.csv')
    return eval_df.sample(topn)
    

In [201]:
def test_range(df, prompt_configs, prompt_prefix, examples, prompt_suffix,
               pre_processing=lambda x:x, post_processing=lambda y:y,
               model='togethercomputer/llama-2-70b', debug=False):
    tweet_idx = []
    answers = []
    model_responses = []
    corrected_model_responses = []
    tweet_txt_list = []
    ap_text = ""
    for idx, row in tqdm(df.iterrows()):
        tweet_idx.append(row['id'])
        fixed_prompt = row['tweet_text'] + "\n"
        tweet_txt_list.append(row['tweet_text'])
        fixed_prompt = pre_processing(fixed_prompt)
        prompt = prompt_prefix + examples + fixed_prompt + prompt_suffix
        answer = row['offense']
        answers.append(answer)
        model_response = call_together_api(prompt, prompt_configs, pre_processing, lambda y:y, model=model, debug=debug)
        corrected_model_response = post_processing(model_response)
        corrected_model_responses.append(corrected_model_response)
        model_responses.append(model_response)
        sleep(1)
    df = pd.DataFrame({'tweet_idx': tweet_idx, 'tweet_text': tweet_txt_list, 'model_responses': model_responses, 'corrected_model_responses':corrected_model_responses, 'offense': answers})
    return df

## Zero Shot Prompt

In [202]:
prompt_prefix_zs = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. 
Your output should only be either "OFF" for offensive, or "NOT" for not offensive.
'''
prompt_examples_zs = "Input Tweet: "
prompt_suffix_zs = "Output: "


In [213]:
# input_string: '@user tweet'
def your_pre_processing_zs(input_string):
    return re.sub(r"@user","", input_string).strip()

def your_post_processing_zs(output_string):
    # using regular expression to find the first consecutive digits in the returned string
    if output_string.strip().lower()[:3]=='off':
        return 1
    elif output_string.strip().lower()[:3]=='not':
        return 0
    else:
        return 0

In [204]:
prompt_config_zs = {'max_tokens': 3,
                'temperature': 0.4,
                'top_k': 50,
                'top_p': 0.7,
                'repetition_penalty': 1,
                'stop': []}

model = 'togethercomputer/llama-2-70b-chat'
print(model)

eval_df = get_eval_df(50)

togethercomputer/llama-2-70b-chat


In [61]:
results_df = test_range(eval_df, prompt_config_zs, prompt_examples_zs, prompt_prefix_zs, prompt_suffix_zs, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
print(results_df)

50it [01:25,  1.72s/it]

    tweet_idx                                         tweet_text  \
0         632  #Demonetisation A dream of many executed by #M...   
1      178287  @user Aren't honourable Judges aware about pre...   
2       79887  @user Muslims harassing Hindus in UK, USA and ...   
3        1062  @user Where are when 2015 floods  came …..when...   
4       34391  INDIAN ARMY CARRIES OUT LARGE FIELD TRAINING E...   
5       65790  is covid-19 still a thing of have they deemed ...   
6      169445  Cricket gods should honour and bless Sikander ...   
7       26597  @user @user Your tweet wasn't worded properly ...   
8       15249  @user @user @user You're just flat wrong about...   
9       75929  @user @user Yes, but the data says whole hindi...   
10      59974  @user Comparison is lame. Biden reduced defici...   
11     181862  @user @user \nIt's a mockery of the system!\nA...   
12      30342  @user Big salute no sir  \nBig big salute Indi...   
13     213424  @user In 1966, Golwalkar, in his 

In [79]:
sum(results_df['corrected_model_responses']==results_df['offense'].values)

29

In [80]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.15999999999999998

In [94]:
test_df = get_test_set()
results_df = test_range(test_df, prompt_config_zs, prompt_examples_zs, prompt_prefix_zs, prompt_suffix_zs, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
results_df.to_csv('zsl_test_results.csv', index=False)

0it [00:00, ?it/s]

851it [23:21,  1.65s/it]


In [96]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.7196    0.7160    0.7178       595
           1     0.3475    0.3516    0.3495       256

    accuracy                         0.6063       851
   macro avg     0.5335    0.5338    0.5336       851
weighted avg     0.6077    0.6063    0.6070       851



In [106]:
joined_df = results_df.set_index("tweet_idx").join(test_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [115]:
cm = joined_df.loc[joined_df.codemixed==1]

In [116]:
print(classification_report(cm['offense_caller'], cm['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.6631    0.6752    0.6691       274
           1     0.3862    0.3733    0.3797       150

    accuracy                         0.5684       424
   macro avg     0.5246    0.5243    0.5244       424
weighted avg     0.5651    0.5684    0.5667       424



In [117]:
mono = joined_df.loc[joined_df.codemixed==0]

In [118]:
print(classification_report(mono['offense_caller'], mono['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.7700    0.7508    0.7603       321
           1     0.2982    0.3208    0.3091       106

    accuracy                         0.6440       427
   macro avg     0.5341    0.5358    0.5347       427
weighted avg     0.6529    0.6440    0.6483       427



## Few Shot Prompting (In Context Learning)
Useful to fix output format

In [228]:
train_df = get_train_df(50)

In [230]:
def create_example(row):
    line1 = "Input Tweet: " + row['tweet_text'] + "\n"
    label = 'OFF' if row['offense']==1 else 'NOT'
    line2 = "Output: " + label + "\n"
    return line1+line2


In [231]:
prompt_examples_icl = ""
for idx,row in train_df.iterrows():
    ex = create_example(row)
    prompt_examples_icl += ex

In [232]:
prompt_examples_icl = prompt_examples_icl + "Input Tweet: "

In [233]:
prompt_prefix_icl = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. Your output should only be either "OFF" for offensive, or "NOT" for not offensive like in the examples below.
'''



prompt_suffix_icl = "Output: "


In [234]:
# results_df = test_range(eval_df, prompt_config_zs, prompt_examples_icl, prompt_prefix_icl, prompt_suffix_icl, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
# print(results_df)

In [146]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.4444444444444445

In [136]:
test_df = get_test_set()
results_df = test_range(test_df, prompt_config_zs, prompt_examples_icl, prompt_prefix_icl, prompt_suffix_icl, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
results_df.to_csv('icl_test_results_50ex.csv', index=False)

0it [00:00, ?it/s]

851it [28:40,  2.02s/it]


In [137]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.9231    0.2218    0.3577       595
           1     0.3460    0.9570    0.5083       256

    accuracy                         0.4430       851
   macro avg     0.6346    0.5894    0.4330       851
weighted avg     0.7495    0.4430    0.4030       851



In [138]:
joined_df = results_df.set_index("tweet_idx").join(test_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [139]:
cm = joined_df.loc[joined_df.codemixed==1]
mono = joined_df.loc[joined_df.codemixed==0]

In [140]:
print(classification_report(cm['offense_caller'], cm['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.8889    0.1752    0.2927       274
           1     0.3892    0.9600    0.5538       150

    accuracy                         0.4528       424
   macro avg     0.6390    0.5676    0.4233       424
weighted avg     0.7121    0.4528    0.3851       424



In [141]:
mono = joined_df.loc[joined_df.codemixed==0]

In [142]:
print(classification_report(mono['offense_caller'], mono['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.9438    0.2617    0.4098       321
           1     0.2988    0.9528    0.4550       106

    accuracy                         0.4333       427
   macro avg     0.6213    0.6073    0.4324       427
weighted avg     0.7837    0.4333    0.4210       427



## Chain of Thought Prompting

In [173]:
off_tweets = [
"Ho skta h kuch baudh to teri tarah dogle bhi nhi hote..balki bahut hmare sc ya dalit bhai hmare sath mandiro me jate h khub dharm ko mante hai snatan me viswas krte h..or kuch gaddar meem walo ka gana gate hai..wo acha h lekin jo tum kar rhe ho wo ghtiya h..jai meem bol ab",
"Kab aana hai ? # Gharwapasi",
"Haa...Ab Umpire Bhi IPL khelega Chutiya Sala Fixer."]

In [174]:
justification_off_tweets = ["This tweet is offensive because it is insulting in nature", "This tweet is offensive due to the the context of Ghar Wapasi which involves religious conversions", "This tweet is offensive as it insults the umpire and uses cuss words."]

In [175]:
non_off_tweets = ["I don't understand why ppl take law in hands...agar chor tha toh police complaint hone thee...bekar me moka diya jata h ke dalit exploitation ho raha h...", "I have of hindus crimes data agints dalit u hater.\
Dalit hater community.\
Do u have data to prove me otherwise and provide data for muslim crime against hindu", "#demonetisation  .. mention bigger scam than this 😂😂",
"Can he himself come to court room in an himachali outfit or in an pure old adivasi dress(no dress)...",
"Just like modi said corruption will end with demonetisation\
15 L will be in everyone's account \
2 million jobs every year \
USA will stand in line for visa etc \
Aap &amp; bjpeee are the two sides of lies &amp; deceit", "Romanticizing open defecation under heavy rain to enjoy the melancholy"]

In [176]:
justification_non_off_tweets = ["This tweet is not offensive as it is against lynching of individuals", "This tweet is not offensive as it just asks for more data", "This tweet is not offensive as it criticizes a government policy which is allowed in a healthy democracy",
                                "This tweet is not offensive as it is in fact showcasing cultural diversity and not containing offensive language",
                                "This tweet is not offensive as even though it is critical of political promises, it simply expresses a perspective on political issues.", "This tweet is not offensive despite the sarcasm, because it is satirical in nature without offending any group in particular"]

In [177]:
import random
random.seed(40)
def create_examples(off_tweets, justification_off_tweets, non_off_tweets, justification_non_off_tweets):
    examples = []

    combined_tweets = list(zip(off_tweets, justification_off_tweets, ['OFF'] * len(off_tweets))) + \
                      list(zip(non_off_tweets, justification_non_off_tweets, ['NOT'] * len(non_off_tweets)))

    random.shuffle(combined_tweets)

    for tweet, justification, label in combined_tweets:
        line1 = "Input Tweet: " + tweet + "\n"
        justification_line = "Justification: " + justification + "\n"
        line2 = "Output: " + label + "\n"
        examples.append(line1 + justification_line + line2)

    return ''.join(examples)

In [178]:
# def create_examples(off_tweets, justification_off_tweets, non_off_tweets, justification_non_off_tweets):
#     examples = ""
#     for i in range(len(off_tweets)):
#         line1 = "Input Tweet: " + off_tweets[i] + "\n"
#         justification_off = "Justification: " + justification_off_tweets[i] + "\n"
#         line2 = "Output: " + 'OFF' + "\n"
#         line3 = "Input Tweet: " + non_off_tweets[i] + "\n"
#         justification_not = "Justification: " + justification_non_off_tweets[i] + "\n"
#         line4 = "Output: " + 'NOT' + "\n"
#         set = line1 + justification_off + line2 + line3 + justification_not + line4
#         examples+=set
    
#     return examples


In [179]:
prompt_examples_cot = create_examples(off_tweets, justification_off_tweets, non_off_tweets, justification_non_off_tweets) + "Input Tweet: "

In [180]:
print(prompt_examples_cot)

Input Tweet: Romanticizing open defecation under heavy rain to enjoy the melancholy
Justification: This tweet is not offensive despite the sarcasm, because it is satirical in nature without offending any group in particular
Output: NOT
Input Tweet: #demonetisation  .. mention bigger scam than this 😂😂
Justification: This tweet is not offensive as it criticizes a government policy which is allowed in a healthy democracy
Output: NOT
Input Tweet: I don't understand why ppl take law in hands...agar chor tha toh police complaint hone thee...bekar me moka diya jata h ke dalit exploitation ho raha h...
Justification: This tweet is not offensive as it is against lynching of individuals
Output: NOT
Input Tweet: I have of hindus crimes data agints dalit u hater.Dalit hater community.Do u have data to prove me otherwise and provide data for muslim crime against hindu
Justification: This tweet is not offensive as it just asks for more data
Output: NOT
Input Tweet: Can he himself come to court room 

In [181]:
prompt_prefix_cot = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. Your output should be a only be either "OFF" for offensive, or "NOT" for not offensive along with a justification for your output like in the examples below.
'''



prompt_suffix_cot = "Justification: "

In [182]:
# input_string: '@user tweet'
def your_pre_processing_cot(input_string):
    return re.sub(r"@user","", input_string).strip()

def your_post_processing_cot(output_string):
    # using regular expression to find the first consecutive digits in the returned string
    if output_string.find("not offensive")!=-1:
        return 0
    else:
        return 1

In [183]:
prompt_config_cot = {'max_tokens': 30,
                'temperature': 0.4,
                'top_k': 50,
                'top_p': 0.7,
                'repetition_penalty': 1,
                'stop': []}

model = 'togethercomputer/llama-2-70b-chat'
print(model)

eval_df = get_eval_df(50)

togethercomputer/llama-2-7b


In [184]:
results_df = test_range(eval_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)
print(results_df)

50it [01:31,  1.84s/it]

    tweet_idx                                         tweet_text  \
0       75787  @user I didnt even know he is from that caste....   
1      178470  @user Have you done any report on lakhs of und...   
2      124492  @user You feel proud when hindu girls are murd...   
3      192196  @user Indian Passport me Ashok chinha ke saath...   
4      120178  @user @user Sanjay, remind me again, which par...   
5      186613  @user Jo bharose Wale emergency lagye hai wo k...   
6      185294  @user It's called summer weather. 1 week it's ...   
7       75963  @user @user @user @user @user @user 1.  You as...   
8       41859  @user ...genesis of the matter, you will mista...   
9      213324  The national flag is a matter of pride for us....   
10     120424  @user Jin kursiyon per abhi Kuchh log dikh rah...   
11      80036  @user Tumhare dukane ache chal rahe he hindu M...   
12     169159  @user @user G bilkul wo zamane gae ab sb teams...   
13       1037  @user 2/2 You may DM me if you li

In [68]:
results_df.to_csv('error_analysis.csv', index=False)

In [192]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [186]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.4210526315789474

In [193]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses']))

              precision    recall  f1-score   support

           0       0.77      0.78      0.77       595
           1       0.47      0.45      0.46       256

    accuracy                           0.68       851
   macro avg       0.62      0.61      0.61       851
weighted avg       0.68      0.68      0.68       851



## Test Set Inference

In [188]:
test_df = get_test_set()

In [191]:
results_df = test_range(test_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)

851it [25:45,  1.82s/it]


In [55]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [56]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses']))

              precision    recall  f1-score   support

           0       0.85      0.75      0.80       595
           1       0.54      0.69      0.61       256

    accuracy                           0.73       851
   macro avg       0.70      0.72      0.70       851
weighted avg       0.76      0.73      0.74       851



In [63]:
results_df.to_csv('test_results.csv', index=False)

In [23]:
results_df = test_range(test_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)

851it [29:50,  2.10s/it]


In [24]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [144]:
results_df = pd.read_csv('test_results.csv')

In [145]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.8498    0.7513    0.7975       595
           1     0.5446    0.6914    0.6093       256

    accuracy                         0.7333       851
   macro avg     0.6972    0.7213    0.7034       851
weighted avg     0.7580    0.7333    0.7409       851



In [146]:
joined_df = results_df.set_index("tweet_idx").join(test_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [147]:
cm = joined_df.loc[joined_df.codemixed==1]
mono = joined_df.loc[joined_df.codemixed==0]

In [148]:
print(classification_report(mono['offense_caller'], mono['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.8715    0.7819    0.8243       321
           1     0.4964    0.6509    0.5633       106

    accuracy                         0.7494       427
   macro avg     0.6840    0.7164    0.6938       427
weighted avg     0.7784    0.7494    0.7595       427



In [149]:
print(classification_report(cm['offense_caller'], cm['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.8235    0.7153    0.7656       274
           1     0.5806    0.7200    0.6429       150

    accuracy                         0.7170       424
   macro avg     0.7021    0.7177    0.7042       424
weighted avg     0.7376    0.7170    0.7222       424



## OOD Set Inference

In [241]:
def get_ood_set():
    ood_df = pd.read_csv('./data/splits/cm_hate_combined.csv')
    return ood_df

In [242]:
ood_df = get_ood_set()

In [243]:
ood_df['tweet_text'] = ood_df['tweet']

In [245]:
model = 'togethercomputer/llama-2-70b'

In [246]:
results_df = test_range(ood_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)

641it [23:39,  2.21s/it]


In [247]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [248]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses']))

              precision    recall  f1-score   support

           0       0.88      0.45      0.60       362
           1       0.56      0.92      0.70       279

    accuracy                           0.66       641
   macro avg       0.72      0.69      0.65       641
weighted avg       0.74      0.66      0.64       641



In [30]:
results_df = test_range(ood_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)

641it [22:11,  2.08s/it]


In [34]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [37]:
results_df.to_csv('ood_results.csv', index=False)

In [65]:
results_df = pd.read_csv('ood_results.csv')

In [67]:
print(classification_report(results_df['offense'], results_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.9179    0.3398    0.4960       362
           1     0.5286    0.9606    0.6819       279

    accuracy                         0.6100       641
   macro avg     0.7233    0.6502    0.5890       641
weighted avg     0.7485    0.6100    0.5769       641



In [68]:
joined_df = results_df.set_index('tweet_idx').join(ood_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [69]:
rgn = joined_df.loc[joined_df['domain']=='religion']
gen = joined_df.loc[joined_df['domain']=='gender']
ori = joined_df.loc[joined_df['domain']=='orientation']

In [226]:
print(classification_report(rgn['offense_caller'], rgn['corrected_model_responses'], digits=4))
print(classification_report(gen['offense_caller'], gen['corrected_model_responses'], digits=4))
print(classification_report(ori['offense_caller'], ori['corrected_model_responses'], digits=4))


              precision    recall  f1-score   support

           0     0.6421    0.7821    0.7052       156
           1     0.4925    0.3267    0.3929       101

    accuracy                         0.6031       257
   macro avg     0.5673    0.5544    0.5490       257
weighted avg     0.5833    0.6031    0.5825       257



## ZSL OOD

In [208]:
results_df = test_range(ood_df, prompt_config_zs, prompt_examples_zs, prompt_prefix_zs, prompt_suffix_zs, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)


641it [17:16,  1.62s/it]


In [209]:
results_df.to_csv('zsl_ood_results.csv', index=False)

In [224]:
joined_df = results_df.set_index('tweet_idx').join(ood_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [225]:
rgn = joined_df.loc[joined_df['domain']=='religion']
gen = joined_df.loc[joined_df['domain']=='gender']
ori = joined_df.loc[joined_df['domain']=='orientation']

In [227]:
print(classification_report(rgn['offense_caller'], rgn['corrected_model_responses'], digits=4))
print(classification_report(gen['offense_caller'], gen['corrected_model_responses'], digits=4))
print(classification_report(ori['offense_caller'], ori['corrected_model_responses'], digits=4))
print(classification_report(joined_df['offense_caller'], joined_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     0.6421    0.7821    0.7052       156
           1     0.4925    0.3267    0.3929       101

    accuracy                         0.6031       257
   macro avg     0.5673    0.5544    0.5490       257
weighted avg     0.5833    0.6031    0.5825       257

              precision    recall  f1-score   support

           0     0.5476    0.7616    0.6371       151
           1     0.4706    0.2520    0.3282       127

    accuracy                         0.5288       278
   macro avg     0.5091    0.5068    0.4827       278
weighted avg     0.5124    0.5288    0.4960       278

              precision    recall  f1-score   support

           0     0.4872    0.6909    0.5714        55
           1     0.3929    0.2157    0.2785        51

    accuracy                         0.4623       106
   macro avg     0.4400    0.4533    0.4250       106
weighted avg     0.4418    0.4623    0.4305       106

              preci

## ICL OOD

In [236]:
results_df = test_range(ood_df, prompt_config_zs, prompt_examples_icl, prompt_prefix_icl, prompt_suffix_icl, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
results_df.to_csv('icl_ood_results_50ex.csv', index=False)

641it [20:53,  1.95s/it]


In [238]:
joined_df = results_df.set_index('tweet_idx').join(ood_df.set_index('id'), lsuffix='_caller', rsuffix='_other')

In [239]:
rgn = joined_df.loc[joined_df['domain']=='religion']
gen = joined_df.loc[joined_df['domain']=='gender']
ori = joined_df.loc[joined_df['domain']=='orientation']

In [240]:
print(classification_report(rgn['offense_caller'], rgn['corrected_model_responses'], digits=4))
print(classification_report(gen['offense_caller'], gen['corrected_model_responses'], digits=4))
print(classification_report(ori['offense_caller'], ori['corrected_model_responses'], digits=4))
print(classification_report(joined_df['offense_caller'], joined_df['corrected_model_responses'], digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.0769    0.1429       156
           1     0.4122    1.0000    0.5838       101

    accuracy                         0.4397       257
   macro avg     0.7061    0.5385    0.3633       257
weighted avg     0.7690    0.4397    0.3162       257

              precision    recall  f1-score   support

           0     0.8000    0.0530    0.0994       151
           1     0.4664    0.9843    0.6329       127

    accuracy                         0.4784       278
   macro avg     0.6332    0.5186    0.3661       278
weighted avg     0.6476    0.4784    0.3431       278

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        55
           1     0.4811    1.0000    0.6497        51

    accuracy                         0.4811       106
   macro avg     0.2406    0.5000    0.3248       106
weighted avg     0.2315    0.4811    0.3126       106

              preci

/Users/shikharrastogi/opt/anaconda3/envs/2590-hw4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shikharrastogi/opt/anaconda3/envs/2590-hw4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shikharrastogi/opt/anaconda3/envs/2590-hw4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa